# RMSD ADCP – Verger-Miralles et al. (2025)

**SWOT enhances small-scale eddy detection in the Mediterranean Sea**

Author: *Elisabet Verger-Miralles*  
Institution: IMEDEA (CSIC-UIB)

Compute RMSD ADCP cross-section vels. vs SWOT and DUACS

In [2]:
import glob 
import pandas as pd
import xarray as xr
import numpy as np
import cmath
import matplotlib.pyplot as plt
from scipy.stats import bootstrap
from scipy.interpolate import interp1d, griddata
from functions import haversine, low_pass_filter

In [3]:
dir_adcp = '../../grl_codes_to_publish_def_swotv2.0.1/data/ADCP/'
dir_SWOT = '../../grl_codes_to_publish_def_swotv2.0.1/data/SWOT/'
dir_duacs = '../../SWOT/imatges/SSH/reprocessed/'
file_swot = 'SWOT_L3_LR_SSH_Expert_502_016_20230426T062612_20230426T071716_v2.0.1.nc'  
file_duacs = 'cmems_obs-sl_eur_phy-ssh_my_allsat-l4-duacs-0.125deg_P1D_20230426.nc'  

Import data, compute cross-section velocities and concatenate the differences:

In [20]:
# Transects to process
transects = [6, 8, 9, 10, 11, 12, 13]
depth_level = 80 

# Load datasets
ds_adcp = xr.open_dataset(dir_adcp + 'adcp_selected_transects_fast_swot.nc')
df_adcp = ds_adcp.to_dataframe().reset_index(drop=False)
ds_swot = xr.open_dataset(dir_SWOT + file_swot)
ds_duacs = xr.open_dataset(dir_duacs + file_duacs)

# Storage for results
rmsd_s_all = []
rmsd_d_all = []
uv_ADCP_cross_all = []
uv_DUACS_cross_all = []
uv_SWOT_cross_all = []

for trans in transects:
    # ADCP data
    lon_adcp = np.array(df_adcp.loc[(df_adcp.transect_id == trans) & (df_adcp.depth == depth_level), 'lon'])[1:]
    lat_adcp = np.array(df_adcp.loc[(df_adcp.transect_id == trans) & (df_adcp.depth == depth_level), 'lat'])[1:]
    u_adcp = df_adcp.loc[(df_adcp.transect_id == trans) & (df_adcp.depth == depth_level), 'u'].values[1:]
    v_adcp = df_adcp.loc[(df_adcp.transect_id == trans) & (df_adcp.depth == depth_level), 'v'].values[1:]

    # Distance calculation
    lat0, lon0 = lat_adcp[0], lon_adcp[0]
    distances = np.array([haversine(lat0, lon0, lat, lon) for lat, lon in zip(lat_adcp, lon_adcp)])
    regular_distances = np.arange(0, distances[-1], 0.25)

    # Interpolation
    interpolator = interp1d(distances, list(zip(lat_adcp, lon_adcp)), axis=0, kind='linear')
    interpolated_lats, interpolated_lons = interpolator(regular_distances).T

    u_interp = interp1d(distances, u_adcp)(regular_distances)
    v_interp = interp1d(distances, v_adcp)(regular_distances)

    # Filtering
    fs = 1 / (regular_distances[1] - regular_distances[0])
    cutoff_distance = 0.067
    u_filt = low_pass_filter(u_interp, cutoff_distance, fs)
    v_filt = low_pass_filter(v_interp, cutoff_distance, fs)

    # Transect angle
    if trans == 6:
        lon1, lat1 = 1.41, 39.65
        lon2, lat2 = 1.58, 40
        dlon = lon1 - lon2
        dlat = lat1 - lat2
        
    else:
        lon1, lat1 = 1.73, 39.8
        lon2, lat2 = 1.3, 39.9
        dlon=lon2-lon1;
        dlat=lat2-lat1;
    

    mean_lat = np.radians((lat1 + lat2) / 2.0)
    angle = np.degrees(np.arctan2(dlat, dlon * np.cos(mean_lat)))

    # Rotate velocities
    uv_cross = ((u_filt + 1j * v_filt) * cmath.exp(-1j * np.radians(angle))).imag

    # SWOT interpolation
    lon_min, lon_max = lon_adcp.min()-0.5, lon_adcp.max()+0.5
    lat_min, lat_max = lat_adcp.min()-0.5, lat_adcp.max()+0.5

    swot = ds_swot.where(
    (ds_swot.longitude >= lon_min) & (ds_swot.longitude <= lon_max) &
    (ds_swot.latitude >= lat_min) & (ds_swot.latitude <= lat_max),
    drop=True)

    u_swot = swot.ugos_filtered.values.flatten()
    v_swot = swot.vgos_filtered.values.flatten()
    lon_swot = swot.longitude.values.flatten()
    lat_swot = swot.latitude.values.flatten()

    swot_mask = ~np.isnan(u_swot)
    u_swot = u_swot[swot_mask]
    v_swot = v_swot[swot_mask]
    lon_swot = lon_swot[swot_mask]
    lat_swot = lat_swot[swot_mask]

    u_swot_i = griddata((lon_swot, lat_swot), u_swot, (interpolated_lons, interpolated_lats), method='cubic')
    v_swot_i = griddata((lon_swot, lat_swot), v_swot, (interpolated_lons, interpolated_lats), method='cubic')
    uv_SWOT_cross = ((u_swot_i + 1j * v_swot_i) * cmath.exp(-1j * np.radians(angle))).imag

    # DUACS interpolation
    lon_duacs, lat_duacs = np.meshgrid(ds_duacs.longitude, ds_duacs.latitude)
    u_duacs = np.array(ds_duacs.ugos).ravel()
    v_duacs = np.array(ds_duacs.vgos).ravel()

    # convert u_duacs and v_duacs nan values to zero
    u_duacs = np.nan_to_num(u_duacs, nan=0.0)
    v_duacs = np.nan_to_num(v_duacs, nan=0.0)
    
    lon_d = lon_duacs.ravel()
    lat_d = lat_duacs.ravel()

    u_duacs_i = griddata((lon_d, lat_d), u_duacs, (interpolated_lons, interpolated_lats), method='cubic')
    v_duacs_i = griddata((lon_d, lat_d), v_duacs, (interpolated_lons, interpolated_lats), method='cubic')
    uv_DUACS_cross = ((u_duacs_i + 1j * v_duacs_i) * cmath.exp(-1j * np.radians(angle))).imag

    # RMSD
    rmsd_s = (uv_cross - uv_SWOT_cross)**2
    rmsd_d = (uv_cross - uv_DUACS_cross)**2

    rmsd_s_all.append(rmsd_s)
    rmsd_d_all.append(rmsd_d)

    uv_ADCP_cross_all.append(uv_cross)
    uv_DUACS_cross_all.append(uv_DUACS_cross)
    uv_SWOT_cross_all.append(uv_SWOT_cross)


# Concatenate all transects
rmsd_s_all = np.concatenate(rmsd_s_all)
rmsd_d_all = np.concatenate(rmsd_d_all)
uv_ADCP_cross_all = np.concatenate(uv_ADCP_cross_all)
uv_DUACS_cross_all = np.concatenate(uv_DUACS_cross_all)
uv_SWOT_cross_all = np.concatenate(uv_SWOT_cross_all)

In [21]:
# Calculate the mean of the concatenated arrays
mean_rmsd_s = np.sqrt(np.nanmean(rmsd_s_all))
mean_rmsd_d = np.sqrt(np.nanmean(rmsd_d_all))

# Calculate the percentage improvement
percent_improvement = 100 * (mean_rmsd_d - mean_rmsd_s) / mean_rmsd_d

In [22]:
mean_rmsd_s * 100 , mean_rmsd_d * 100, percent_improvement

(np.float64(8.91665848093341),
 np.float64(13.814783930644248),
 np.float64(35.455679034152034))

## BOOTSTRAP

DUACS


In [7]:
# Define RMSD function that takes two separate 1D arrays
def rmsd(drifter, swot, axis=0):
    """Compute Root Mean Square Deviation (RMSD)."""
    diff = drifter - swot
    return np.sqrt(np.nanmean(diff**2, axis=axis))

# Combine data into a tuple without reshaping
data = (uv_ADCP_cross_all, uv_DUACS_cross_all)

# Perform bootstrap resampling
res = bootstrap(
    data, 
    statistic=rmsd, 
    n_resamples=1000, 
    confidence_level=0.95, 
    method='BCa',  # Bias-Corrected and Accelerated bootstrap method
    paired=True,  # Since we compare paired velocity values
    random_state=42  # For reproducibility
)

# Print results
print(f"RMSD: {rmsd(uv_ADCP_cross_all, uv_DUACS_cross_all):.4f}")

print(f"95% Confidence Interval: {res.confidence_interval}")


RMSD: 0.1388
95% Confidence Interval: ConfidenceInterval(low=np.float64(0.13601159921957126), high=np.float64(0.14227916117771314))


In [8]:
# low confidence interval
low_ci = res.confidence_interval[0]
high_ci = res.confidence_interval[1]
low_ci, high_ci

(np.float64(0.13601159921957126), np.float64(0.14227916117771314))

In [9]:
0.1388 - low_ci, high_ci - 0.1388 # longitude of the error bar

(np.float64(0.002788400780428746), np.float64(0.0034791611777131304))

In [10]:
(((0.1388 - low_ci)*100) + ((high_ci - 0.1388)*100))/2

np.float64(0.3133780979070938)

SWOT


In [11]:
data = (uv_ADCP_cross_all, uv_SWOT_cross_all)

# Perform bootstrap resampling
res = bootstrap(
    data, 
    statistic=rmsd, 
    n_resamples=1000, 
    confidence_level=0.95, 
    method='BCa',  # Bias-Corrected and Accelerated bootstrap method
    paired=True,  # Since we compare paired velocity values
    random_state=42  # For reproducibility
)

# Print results
print(f"RMSD: {rmsd(uv_ADCP_cross_all, uv_SWOT_cross_all):.4f}")

print(f"95% Confidence Interval: {res.confidence_interval}")

RMSD: 0.0888
95% Confidence Interval: ConfidenceInterval(low=np.float64(0.08517299453592632), high=np.float64(0.09241633280744484))


In [12]:
# low confidence interval
low_ci = res.confidence_interval[0]
high_ci = res.confidence_interval[1]
low_ci, high_ci

(np.float64(0.08517299453592632), np.float64(0.09241633280744484))

In [13]:
0.0888 - low_ci, high_ci - 0.0888 # longitude of the error bar

(np.float64(0.003627005464073685), np.float64(0.003616332807444833))

In [14]:
(((0.0888 - low_ci)*100) + ((high_ci - 0.0888)*100))/2

np.float64(0.3621669135759259)

In [15]:
percent_improvement = 100 * (0.1388 - 0.0888) / 0.1388
percent_improvement

36.023054755043226